Model Ideas:

Transfer Learning:
- Vision Transformer (https://github.com/google-research/vision_transformer)
- Vision Transformer Adapter (https://github.com/czczup/vit-adapter)
- EfficientNetV2 ()
- ConvNeXt
- ResNET50
- Novel arch using cosine similarity instead of convolution (see: https://github.com/Ritz-19/cosine_sim_NN)


Data pipeline:
- unzip datafiles
- 

In [2]:
conda install scikit-image

Solving environment: ...working... done

## Package Plan ##

  environment location: c:\Users\Derek\AppData\Local\R-MINI~1\envs\tf-gpu-3

  added / updated specs:
    - scikit-image


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blosc-1.21.0               |       h19a0ad4_0         139 KB
    brotli-1.0.9               |       ha925a31_2         332 KB
    bzip2-1.0.8                |       he774522_0         113 KB
    certifi-2022.5.18.1        |   py39haa95532_0         157 KB
    cfitsio-3.470              |       he774522_6         512 KB
    charls-2.2.0               |       h6c2663c_0          82 KB
    cloudpickle-2.0.0          |     pyhd3eb1b0_0          32 KB
    cytoolz-0.11.0             |   py39h2bbff1b_0         294 KB
    dask-core-2022.5.0         |   py39haa95532_0         1.6 MB
    fsspec-2022.3.0            |   py39haa95532_0         208 KB
    imagecodecs-2021.8



==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.12.0

Please update conda by running

    $ conda update -n base -c defaults conda




In [ ]:
import numpy as np
import os.path
from PIL import Image
from glob import glob
from skimage.transform import resize

Read data files:

In [1]:
# training
training_x_path = './datasets/training/training/images/*.tif'
training_y_path = './datasets/training/training/1st_manual/*.gif'
# testing
test_x_path = './datasets/test/test/images/*.tif'
test_y_path = './datasets/test/test/1st_manual/*.gif'
test_mask_path = './datasets/test/test/mask/*.gif'

data_paths = [training_x_path, training_y_path, test_x_path, test_y_path, test_mask_path]

# shape
data_shape = (576, 576)

In [ ]:
def read_input(path):

    if path.find('2nd') > 0 and path.find('DRIVE') > 0:
        x = np.array(Image.open(path)) / 1.
    else:
        x = np.array(Image.open(path)) / 255.
    if x.shape[-1] == 3:
        return x
    else:
        return x[..., np.newaxis]


def preprocessData(data_path, dataset):
    
    global data_shape

    data_path = list(sorted(glob(data_path)))

    if data_path[0].find('mask') > 0:
        return np.array([read_input(image_path) for image_path in data_path])
    else:
        return np.array([resize(read_input(image_path), data_shape) for image_path in data_path])